# Webscrape demo 

Accompanies the Data Science road show.

Intended to serve as an example of accessability of data on the internet.

Photos section does not work properly: the property photos are in a different class from the property info and I was unable to easily match them. I may have overlooked something dead simple. So this is for demo purpose only

Code snippets from Edo van Uitert were used

In [134]:
import warnings
warnings.filterwarnings("ignore")
import urllib3
from bs4 import BeautifulSoup
import pandas as pd
#import geopy

http = urllib3.PoolManager()

In [155]:
def ScrapePageToDf2(soup):
    nproperty=len(soup.find_all("div", attrs={"class": "property-info"}))

    list_address=[""]*nproperty
    list_zip=[""]*nproperty
    list_price=[0]*nproperty

    list_type=[""]*nproperty
    list_rooms=[""]*nproperty
    list_area=[""]*nproperty

    i=0
    for section in soup.find_all("div", attrs={"class": "property-info"}):
        
        try:
            list_address[i]=section.find("h2", attrs={"class": "property-address-street"}).get_text()
        except AttributeError:
            list_address[i]=""
        
        try:
            list_zip[i]=section.find("div", attrs={"class": "property-address-zipcity"}).get_text().split(",")[0]
        except AttributeError:
            list_zip[i]=""     
            
        try:
            list_price[i]=section.find("div", attrs={"class": "property-price"}).get_text().split(" ")[1]
        except AttributeError:
            list_price[i]=""     
            
        checklen=len(list(section.find_all("div", attrs={"class": "property-feature"})))
        
        try:
            checklen=len(list(section.find_all("div", attrs={"class": "property-feature"})))
            if(checklen==3):
                list_type[i]=section.find_all("div", attrs={"class": "property-feature"})[0].get_text()
                list_rooms[i]=section.find_all("div", attrs={"class": "property-feature"})[1].get_text().split(" ")[0]
                list_area[i]=section.find_all("div", attrs={"class": "property-feature"})[2].get_text().split("ᵐ")[0]

        except AttributeError:
            list_type[i]=""  
            list_rooms[i]=""  
            list_area[i]=""  

        i+=1

    feat = [('address', list_address),
         ('zip', list_zip),
         ('price', list_price),
         ('type', list_type),
         ('rooms', list_rooms),
         ('area', list_area),
                  
         ]
    
    df = pd.DataFrame.from_items(feat)
    
    return df

In [174]:
import tqdm
firstpage='https://www.jaap.nl/koophuizen/utrecht/utrecht/amersfoort'

response = http.request('GET', firstpage)

page_stat=response.status

if(page_stat==200):
    print("Page 1 done")
    soup = BeautifulSoup(response.data)
    npage=soup.find("span", attrs={"class": "page-info"}).get_text().strip().split(" ")[3]
    df_all=ScrapePageToDf2(soup)

print("Length df:",len(df_all))

for i in range(int(npage)-1):
    nextpage=firstpage+"/p"+str(i+2)
    print(nextpage)
    response = http.request('GET', nextpage)
    page_stat=response.status
    if(page_stat==200):
        soup = BeautifulSoup(response.data)
        df_part=ScrapePageToDf2(soup)
        
        df_all=pd.concat([df_all,df_part])
        print("Page %d done" %(i+2))
        
        print("Length df:",len(df_all))
        
df_all=df_all.reset_index(drop=True)

Page 1 done
Length df: 30
https://www.jaap.nl/koophuizen/utrecht/utrecht/amersfoort/p2
Page 2 done
Length df: 60
https://www.jaap.nl/koophuizen/utrecht/utrecht/amersfoort/p3
Page 3 done
Length df: 90
https://www.jaap.nl/koophuizen/utrecht/utrecht/amersfoort/p4
Page 4 done
Length df: 120
https://www.jaap.nl/koophuizen/utrecht/utrecht/amersfoort/p5
Page 5 done
Length df: 150
https://www.jaap.nl/koophuizen/utrecht/utrecht/amersfoort/p6
Page 6 done
Length df: 180
https://www.jaap.nl/koophuizen/utrecht/utrecht/amersfoort/p7
Page 7 done
Length df: 210
https://www.jaap.nl/koophuizen/utrecht/utrecht/amersfoort/p8
Page 8 done
Length df: 240
https://www.jaap.nl/koophuizen/utrecht/utrecht/amersfoort/p9
Page 9 done
Length df: 270
https://www.jaap.nl/koophuizen/utrecht/utrecht/amersfoort/p10
Page 10 done
Length df: 300
https://www.jaap.nl/koophuizen/utrecht/utrecht/amersfoort/p11
Page 11 done
Length df: 330
https://www.jaap.nl/koophuizen/utrecht/utrecht/amersfoort/p12
Page 12 done
Length df: 360
ht

In [157]:
df_all

,address,zip,price,type,rooms,area
0,Diamantweg 81,3817 GJ,550.000,Eengezinswoning,6,165 m²
1,Daam Fockemalaan 88,3818 KH,1.345.000,Eengezinswoning,8,351 m²
2,Zielhorsterlaan 17,3822 AK,300.000,Hoekwoning,5,100 m²
3,Schuilenburgerplein 12,3816 TE,345.000,Eengezinswoning,7,183 m²
4,Puntenburgerlaan 58,3812 CG,450.000,Eengezinswoning,5,155 m²
5,Jakob Niewegpad 2,3812 GW,485.000,Eengezinswoning,4,129 m²
6,Amerikaring 2,3823 HT,400.000,Eengezinswoning,7,155 m²
7,Sprengenberg 18,3825 GB,375.000,Tussenwoning,7,176 m²
8,Scheltussingel 64,3814 BH,300.000,Eengezinswoning,5,118 m²
9,Sprengenberg 184,3825 GZ,359.000,Tussenwoning,6,154 m²


In [158]:
def ScrapePageToDf2(soup):
    nproperty=len(soup.find_all("div", attrs={"class": "property-info"}))

    list_address=[""]*nproperty
    list_zip=[""]*nproperty
    list_price=[0]*nproperty

    list_type=[""]*nproperty
    list_rooms=[""]*nproperty
    list_area=[""]*nproperty
    list_photo=[""]*nproperty

    i=0
    for section in soup.find_all("div", attrs={"class": "property-info"}):
        
        try:
            list_address[i]=section.find("h2", attrs={"class": "property-address-street"}).get_text()
        except AttributeError:
            list_address[i]="" 
        
        try:
            list_zip[i]=section.find("div", attrs={"class": "property-address-zipcity"}).get_text().split(",")[0]
        except AttributeError:
            list_zip[i]=""     
            
        try:
            list_price[i]=section.find("div", attrs={"class": "property-price"}).get_text().split(" ")[1]
        except AttributeError:
            list_price[i]=""     
            
        checklen=len(list(section.find_all("div", attrs={"class": "property-feature"})))
        
        try:
            checklen=len(list(section.find_all("div", attrs={"class": "property-feature"})))
            if(checklen==3):
                list_type[i]=section.find_all("div", attrs={"class": "property-feature"})[0].get_text()
                list_rooms[i]=section.find_all("div", attrs={"class": "property-feature"})[1].get_text().split(" ")[0]
                list_area[i]=section.find_all("div", attrs={"class": "property-feature"})[2].get_text().split("ᵐ")[0]

        except AttributeError:
            list_type[i]=""  
            list_rooms[i]=""  
            list_area[i]=""  
            
    for section in soup.find_all("div", attrs={"class": "property-photo"}):
        try:
            list_photo[i]=section.img["src"]
            
        except AttributeError:
            list_photo[i]=""
        
        i+=1

    feat = [('address', list_address),
         ('zip', list_zip),
         ('price', list_price),
         ('type', list_type),
         ('rooms', list_rooms),
         ('area', list_area),
         ('photo', list_photo)
                  
         ]
    
    df = pd.DataFrame.from_items(feat)
    
    return df

In [183]:
firstpage='https://www.jaap.nl/koophuizen/utrecht/utrecht/amersfoort'

response = http.request('GET', firstpage)

page_stat=response.status

if(page_stat==200):
    print("Page 1 done")
    soup = BeautifulSoup(response.data)
    npage=soup.find("span", attrs={"class": "page-info"}).get_text().strip().split(" ")[3]
    df_all=ScrapePageToDf2(soup)

print("Length df:",len(df_all))

for i in range(int(npage)-1):
    nextpage=firstpage+"/p"+str(i+2)
    print(nextpage)
    response = http.request('GET', nextpage)
    page_stat=response.status
    if(page_stat==200):
        soup = BeautifulSoup(response.data)
        df_part=ScrapePageToDf2(soup)
        
        df_all=pd.concat([df_all,df_part])
        print("Page %d done" %(i+2))
        
        print("Length df:",len(df_all))
        
df_all=df_all.reset_index(drop=True)
mask = df_all.address != ''
df_all = df_all[mask]

Page 1 done
Length df: 30
https://www.jaap.nl/koophuizen/utrecht/utrecht/amersfoort/p2
Page 2 done
Length df: 60
https://www.jaap.nl/koophuizen/utrecht/utrecht/amersfoort/p3
Page 3 done
Length df: 90
https://www.jaap.nl/koophuizen/utrecht/utrecht/amersfoort/p4
Page 4 done
Length df: 120
https://www.jaap.nl/koophuizen/utrecht/utrecht/amersfoort/p5
Page 5 done
Length df: 150
https://www.jaap.nl/koophuizen/utrecht/utrecht/amersfoort/p6
Page 6 done
Length df: 180
https://www.jaap.nl/koophuizen/utrecht/utrecht/amersfoort/p7
Page 7 done
Length df: 210
https://www.jaap.nl/koophuizen/utrecht/utrecht/amersfoort/p8
Page 8 done
Length df: 240
https://www.jaap.nl/koophuizen/utrecht/utrecht/amersfoort/p9
Page 9 done
Length df: 270
https://www.jaap.nl/koophuizen/utrecht/utrecht/amersfoort/p10
Page 10 done
Length df: 300
https://www.jaap.nl/koophuizen/utrecht/utrecht/amersfoort/p11
Page 11 done
Length df: 330
https://www.jaap.nl/koophuizen/utrecht/utrecht/amersfoort/p12
Page 12 done
Length df: 360
ht

In [184]:
df_all

,address,zip,price,type,rooms,area,photo
0,Kamsalamander 13,3824 VM,369.000,Geschakelde woning,5,121 m²,//5.sjaap.nl/jaap/img/small/E65/E6577D2C305D8C4DFC6D19444E53F0C0.jpg
30,Bever 41,3824 VS,485.000,Penthouse,3,156 m²,//sjaap.nl/jaap/img/small/9E4/9E436C3B10AD243BFD8787F1C26A6AD0.jpg
60,Van Obdamstraat 60,3814 TD,259.000,Appartement,3,87 m²,//4.sjaap.nl/jaap/img/small/30A/30AC510EF319228D7AFA8C59ED472E00.jpg
90,Balistraat 18,3818 AK,615.000,Woning,5,142 m²,//1.sjaap.nl/jaap/img/small/D3C/D3C758E974761790AF817D4BDD5EB5C9.jpg
120,Hinderstein 30,3813 RM,op,Appartement,3,74 m²,//4.sjaap.nl/jaap/img/small/AB6/AB6A047ED4C8A4FD2C8D0A635999C695.jpg
150,Gerard Doustraat 13,3817 RR,319.000,Eengezinswoning,4,90 m²,//2.sjaap.nl/jaap/img/small/C29/C29AD7B30EC04B20127F4260F36C2DFE.jpg
180,Paasberg 9,3825 EK,675.000,Vrijstaande woning,5,185 m²,//4.sjaap.nl/jaap/img/small/599/5999A9C2890A5225B7C6064608156906.jpg
210,Keesomstraat 43 B,3817 JZ,200.000,Appartement,3,62 m²,//sjaap.nl/jaap/img/small/1D9/1D92052A0EE1972AB79A32FC06BDA9C0.jpg
240,Wessel Ilckenstraat 62,3813 ZB,op,Tussenwoning,1,92 m²,//sjaap.nl/jaap/img/small/447/4478D15B12B2349CE0A71D68D7F63BC0.jpg
270,Meerval,3824 JP,250.000,Maisonette,4,115 m²,//sjaap.nl/jaap/img/small/F02/F02DA55EA1EE9E39F31DD742FE1E2BC1.jpg


In [185]:
from IPython.display import Image, HTML
pd.set_option('display.max_colwidth', -1)

def path_to_image_html(path):
    return '<img src="'+ path + '"/>'

HTML(df_all.to_html(escape=False ,formatters=dict(photo=path_to_image_html)))


,address,zip,price,type,rooms,area,photo
0,Kamsalamander 13,3824 VM,369.000,Geschakelde woning,5,121 m²,
30,Bever 41,3824 VS,485.000,Penthouse,3,156 m²,
60,Van Obdamstraat 60,3814 TD,259.000,Appartement,3,87 m²,
90,Balistraat 18,3818 AK,615.000,Woning,5,142 m²,
120,Hinderstein 30,3813 RM,op,Appartement,3,74 m²,
150,Gerard Doustraat 13,3817 RR,319.000,Eengezinswoning,4,90 m²,
180,Paasberg 9,3825 EK,675.000,Vrijstaande woning,5,185 m²,
210,Keesomstraat 43 B,3817 JZ,200.000,Appartement,3,62 m²,
240,Wessel Ilckenstraat 62,3813 ZB,op,Tussenwoning,1,92 m²,
270,Meerval,3824 JP,250.000,Maisonette,4,115 m²,


In [186]:
from IPython.display import Image, HTML
pd.set_option('display.max_colwidth', -1)

def path_to_image_html(path):
    return '<img src="'+ path + '"/>'

HTML(df_all.to_html(escape=False ,formatters=dict(photo=path_to_image_html)))

section = soup.findAll("div", attrs={"class": "property-photo"})
l = len(soup.findAll("div", attrs={"class": "property-photo"}))
photos = [""]
for i in range(0,l):
    photos.append(section[i].img["src"])
df_ = pd.DataFrame()
df_['foto']=photos
HTML(df_.to_html(escape=False ,formatters=dict(foto=path_to_image_html)))

,foto
0,
1,
2,
3,
4,
5,


In [19]:
http = urllib3.PoolManager()

In [94]:
import pandas as pd
from IPython.display import Image, HTML

def path_to_image_html(path):
    '''
     This function essentially convert the image url to 
     '<img src="'+ path + '"/>' format. And one can put any
     formatting adjustments to control the height, aspect ratio, size etc.
     within as in the below example. 
    '''

    return '<img src="'+ path + '" style=max-height:124px;"/>'

HTML(df_all.to_html(escape=False ,formatters=dict(photo=path_to_image_html)))

In [20]:
house = soup.findAll("div", attrs={"class": "property-photo"})[0]
house.img["src"]


'//5.sjaap.nl/jaap/img/small/E65/E6577D2C305D8C4DFC6D19444E53F0C0.jpg'